### Pendahuluan

[Keras](https://github.com/keras-team/keras/blob/master/README.md) memiliki banyak model *deep learning* untuk klasifikasi gambar yang siap pakai. Dengan mengatur parameter ``weights='imagenet'``, *weight* hasil *training* berhari-hari pada jutaan gambar dari dataset ImageNet otomatis di*download*. Tanpa kita *training* di PC sendiri, model dapat kita masukkan gambar sembarang untuk diklasifikasi.

Jumlah kelas dataset ImageNet sangat banyak, ada 1000. Perhatikan sampel berikut. 

>#### ImageNet [classes.txt](https://github.com/xmartlabs/caffeflow/blob/master/examples/imagenet/imagenet-classes.txt)
>... head cabbage broccoli cauliflower zucchini, courgette spaghetti squash acorn squash butternut squash cucumber, cuke artichoke, globe artichoke bell pepper cardoon mushroom Granny Smith strawberry orange lemon fig pineapple, ananas **banana** jackfruit, jak, jack custard apple pomegranate ...

Model mampu membedakan berbagai sayur dan buah. Akan tetapi, model tidak cukup spesifik untuk membedakan varian dari setiap sayur atau buah dalam perbendaharaannya. Model akan memprediksi, misalnya, 'orange' ketika dimasukkan gambar jeruk mandarin maupun jeruk sunkist. Pada artikel ini kita akan membuat model yang khusus membedakan antara **pisang tanduk** dan **pisang kepok**. Kita akan menggunakan salah satu model yang disediakan Keras, DenseNet121.

### DenseNet

DenseNet models {121, 169, 201}, with weights pre-trained on ImageNet.

This model and can be built both with 'channels_first' data format (channels, height, width) or 'channels_last' data format (height, width, channels).

The default input size for this model is 224x224.

```
keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

```

#### Arguments
+ blocks: numbers of building blocks for the four dense layers.
+ include_top: whether to include the fully-connected layer at the top of the network.
+ weights: one of None (random initialization), 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded.
+ input_tensor: optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.
+ input_shape: optional shape tuple, only to be specified if include_top is False (otherwise the input shape has to be (224, 224, 3) (with 'channels_last' data format) or (3, 224, 224) (with 'channels_first' data format). It should have exactly 3 inputs channels, and width and height should be no smaller than 32. E.g. (200, 200, 3) would be one valid value.
+ pooling: optional pooling mode for feature extraction when include_top is False. None means that the output of the model will be the 4D tensor output of the last convolutional layer. avg means that global average pooling will be applied to the output of the last convolutional layer, and thus the output of the model will be a 2D tensor. max means that global max pooling will be applied.
+ classes: optional number of classes to classify images into, only to be specified if include_top is True, and if no weights argument is specified.


### Imports

Import secukupnya :)

In [1]:
from keras.applications import densenet
from keras.preprocessing import image
from keras.applications.densenet import preprocess_input, decode_predictions
import numpy as np
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
import os

Using TensorFlow backend.


### Mengumpulkan Gambar Pisang

Dataset khusus gambar pisang tanduk maupun pisang kepok agak sulit untuk ditemukan. Untungnya kita hanya memerlukan sedikit data untuk *transfer learning*, sehingga untuk mencari di Google Image lima puluh gambar pisang rasanya masih manusiawi. Untuk sedikit meringankan pekerjaan (kita harus memverifikasi hasil download secara manual), kita gunakan [google-image-download](https://github.com/hardikvasa/google-images-download).

Simpan gambar-gambar pisang sehingga struktur folder kita menjadi seperti berikut (andaikan kita punya 50 gambar pisang kepok dan 50 pisang tanduk.

    pisang-train/
        pisang-kepok/
            kepok-pisang(1).jpg
            ...
            kepok-manis(45).jpg        
        pisang-tanduk/
            pisang-tanduk(1).jpg
            ...
            tanduk(45).jpg
    pisang-test/
        kepok001.jpg
        kepok002.jpg
        ...
        kepok005.jpg
        tanduk001.jpg
        tanduk002.jpg
        ...
        tanduk005.jpg

Catatan: penamaan data *train* bebas, sedangkan nama file data *test* sebaiknya mengandung kelasnya untuk mempermudah membaca hasil prediksi nanti.

In [2]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory('pisang-train/', 
                                                    target_size=(224,224), 
                                                    color_mode='rgb', 
                                                    batch_size=32, 
                                                    class_mode='categorical', 
                                                    shuffle=True)
category_dict = train_generator.class_indices
print(category_dict)

Found 100 images belonging to 2 classes.
{'pisang-kepok': 0, 'pisang-tanduk': 1}


In [3]:
number_of_classes = len(category_dict)

base_model = densenet.DenseNet121(weights='densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5', 
                                  include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
preds = Dense(number_of_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=preds)

# Print the updated layer names.
# for i,layer in enumerate(model.layers): print(i,layer.name)

# Set the first n_freeze layers of the network to be non-trainable.
n_freeze = 300
for layer in model.layers[:n_freeze]:
    layer.trainable=False
for layer in model.layers[n_freeze:]:
    layer.trainable=True

Catatan: Jumlah layer dapat diketahui dengan ```print(len(model.layers))```. 

Perintah terakhir mengatur supaya 300 layer pertama di*freeze*, yakni hanya layer ke 301 sampai layer sebelum *output* saja yang di*train* parameternya. Jika mau, kita juga dapat mengatur supaya hanya layer *output* saja yang di*train* (murni *transfer learning*) seperti berikut.

    for layer in model.layers:
        layer.trainable=False

In [4]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

step_size_train = train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=step_size_train, 
                    epochs=10)

Epoch 1/10
3/3 [==============================] - 24s 8s/step - loss: 1.0033 - acc: 0.5833
Epoch 2/10
3/3 [==============================] - 13s 4s/step - loss: 0.6711 - acc: 0.7096
Epoch 3/10
3/3 [==============================] - 13s 4s/step - loss: 0.3510 - acc: 0.8534
Epoch 4/10
3/3 [==============================] - 12s 4s/step - loss: 0.1326 - acc: 0.9426
Epoch 5/10
3/3 [==============================] - 18s 6s/step - loss: 0.0367 - acc: 0.9896
Epoch 6/10
3/3 [==============================] - 13s 4s/step - loss: 0.0103 - acc: 1.0000
Epoch 7/10
3/3 [==============================] - 12s 4s/step - loss: 0.0034 - acc: 1.0000
Epoch 8/10
3/3 [==============================] - 12s 4s/step - loss: 0.0620 - acc: 0.9770
Epoch 9/10
3/3 [==============================] - 17s 6s/step - loss: 0.0065 - acc: 1.0000
Epoch 10/10
3/3 [==============================] - 13s 4s/step - loss: 0.0116 - acc: 0.9885


In [5]:
# Without transfer learning.
default_model = densenet.DenseNet121(weights='densenet121_weights_tf_dim_ordering_tf_kernels.h5')

```decode_predictions``` dapat digunakan untuk membuat keluaran model (*array* 1000 kelas) menjadi *human-readable*.
```
prediction = model.predict(x)
keras.applications.densenet.decode_predictions(prediction, top=2)
```
Kebutuhan pengujian model kita dapat diimplementasi dengan *get key by value in dictionary*. (Saya kesulitan mencari *method* serupa yang lebih generik, yakni dapat digunakan untuk *transfer learning* dengan jumlah kelas ≠ 1000.)

In [6]:
test_path = 'pisang-test/'

for directory in os.listdir(test_path):
    
    # Load image.
    img_path = test_path+directory
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    default_preds = default_model.predict(x)

    # Printing results.

    # Default 1000 classes (without transfer learning).
    print(f"Without Transfer Learning Top-2 [{directory}]: \n{decode_predictions(default_preds, top=2)[0]}\n")

    # Print transfer learning model top-1
    confidence_array = preds[0]
    index_max = np.argmax(confidence_array)

    # Get KEY (category) by VALUE (index_max) in dictionary
    # mydict = {'george':16,'amber':19}
    # print(list(mydict.keys())[list(mydict.values()).index(16)]) # Example in one line.

    category_names = category_dict.keys()
    category_values = category_dict.values()
    category_at_index = list(category_values).index(index_max)
    category_max = list(category_names)[category_at_index]

    print(f"\nWith Transfer Learning [{directory}]: \nTop-1 (confidence)\n{category_max} ({max(confidence_array)*100}%)")

    # Print transfer learning model all classes
    print("\nClass (confidence)")

    for category in category_dict:
        category_index = category_dict[category]
        value = confidence_array[category_index] * 100
        print(f"{category} ({value}%)")

    print("\n============================\n")


Without Transfer Learning Top-2 [kepok008.jpg]: 
[('n07753592', 'banana', 0.50477576), ('n07760859', 'custard_apple', 0.301058)]


With Transfer Learning [kepok008.jpg]: 
Top-1 (confidence)
pisang-kepok (100.0%)

Class (confidence)
pisang-kepok (100.0%)
pisang-tanduk (1.496673784906477e-07%)


Without Transfer Learning Top-2 [tanduk006.jpg]: 
[('n04136333', 'sarong', 0.24845652), ('n04417672', 'thatch', 0.21268867)]


With Transfer Learning [tanduk006.jpg]: 
Top-1 (confidence)
pisang-tanduk (99.99977350234985%)

Class (confidence)
pisang-kepok (0.0002311575144631206%)
pisang-tanduk (99.99977350234985%)


Without Transfer Learning Top-2 [kepok015.jpg]: 
[('n03388043', 'fountain', 0.15597062), ('n07753592', 'banana', 0.104576446)]


With Transfer Learning [kepok015.jpg]: 
Top-1 (confidence)
pisang-tanduk (56.99564218521118%)

Class (confidence)
pisang-kepok (43.004366755485535%)
pisang-tanduk (56.99564218521118%)


Without Transfer Learning Top-2 [kepok016.jpg]: 
[('n07753592', 'banana',

Without Transfer Learning Top-2 [kepok010.jpg]: 
[('n07754684', 'jackfruit', 0.48873183), ('n07753592', 'banana', 0.1798069)]


With Transfer Learning [kepok010.jpg]: 
Top-1 (confidence)
pisang-kepok (100.0%)

Class (confidence)
pisang-kepok (100.0%)
pisang-tanduk (1.3649805244431157e-09%)


Without Transfer Learning Top-2 [kepok004.jpg]: 
[('n07753592', 'banana', 0.40068734), ('n07753275', 'pineapple', 0.13428028)]


With Transfer Learning [kepok004.jpg]: 
Top-1 (confidence)
pisang-kepok (100.0%)

Class (confidence)
pisang-kepok (100.0%)
pisang-tanduk (2.765926510051031e-06%)


Without Transfer Learning Top-2 [kepok020.jpg]: 
[('n07753592', 'banana', 0.9886154), ('n07760859', 'custard_apple', 0.0020554317)]


With Transfer Learning [kepok020.jpg]: 
Top-1 (confidence)
pisang-kepok (100.0%)

Class (confidence)
pisang-kepok (100.0%)
pisang-tanduk (4.228452310517383e-08%)


Without Transfer Learning Top-2 [tanduk004.jpg]: 
[('n07753592', 'banana', 0.9968892), ('n07717410', 'acorn_squash',

Catatan: Hasil yang lebih dulu di*print* adalah yang lebih dulu terbaca oleh ```os.listdir()```.

In [7]:
for directory in os.listdir(test_path):
    print(directory, end=' ')

kepok008.jpg tanduk006.jpg kepok015.jpg kepok016.jpg kepok024.jpg kepok018.jpg kepok003.jpg kepok011.jpg tanduk007.jpg tanduk005.jpg tanduk002.jpg kepok001.jpg kepok019.jpg tanduk003.jpg kepok009.jpg kepok002.jpg kepok007.jpg tanduk001.jpg kepok014.jpg kepok021.jpg kepok017.jpg kepok005.jpg kepok013.jpg kepok012.jpg kepok022.jpg kepok010.jpg kepok004.jpg kepok020.jpg tanduk004.jpg kepok023.jpg kepok006.jpg 

### Selesai!

Sesuai motivasi dibuatnya Keras sendiri "*easy and fast prototyping*", notebook ini dibuat generik. Dengan mengulangi langkah yang sama, misalnya, **mengumpulkan gambar berbagai jenis apel**, kita dapat mengklasifikasi jenis apel dengan belasan hingga puluhan gambar saja. (Tentunya apabila data yang dimiliki sedikit, kita harus lebih berinvestasi pada eksperimentasi beberapa variabel seperti jumlah layer yang di*freeze*, ukuran Dense layer, dan sebagainya.)